In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
num_batches = 1000
batch_size = 50
learning_rate = 0.001

dataset = tfds.load("tf_flowers", split=tfds.Split.TRAIN, as_supervised=True)
dataset = dataset.map(lambda img, label: (tf.image.resize(img, [224, 224]) / 255.0, label)).shuffle(1024).batch(32)
model = tf.keras.applications.MobileNetV2(weights=None, classes=5)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
for images, labels in dataset:
    with tf.GradientTape() as tape:
        labels_pred = model(images)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=labels, y_pred=labels_pred)
        loss = tf.reduce_mean(loss)
        print("loss %f" % loss.numpy())
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables))

In [ ]:
import numpy as np
image = np.array([[
    [0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 2, 1, 0],
    [0, 0, 2, 2, 0, 1, 0],
    [0, 1, 1, 0, 2, 1, 0],
    [0, 0, 2, 1, 1, 0, 0],
    [0, 2, 1, 1, 2, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]
]], dtype=np.float32)

In [ ]:
image = np.expand_dims(image, axis=-1)
W = np.array([[
    [ 0, 0, -1], 
    [ 0, 1, 0 ], 
    [-2, 0, 2 ]
]], dtype=np.float32)
b = np.array([1], dtype=np.float32)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(
        filters=1,
        kernel_size=[3,3],
        kernel_initializer=tf.constant_initializer(W),
        bias_initializer=tf.constant_initializer(b)
    )
])

In [ ]:
# this operation returns a tensor of the same type with all dimensions of size 1 removed
output = model(image)
print(tf.squeeze(output))

In [2]:
class DataLoader():
    def __init__(self):
        path = tf.keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
        with open(path, encoding='utf-8') as f:
            self.raw_text = f.read().lower()
        self.chars = sorted(list(set(self.raw_text)))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        #print(self.char_indices)
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
        self.text = [self.char_indices[c] for c in self.raw_text]
        #print(self.text)
    def get_batch(self, seq_length, batch_size):
        seq = []
        next_char = []
        for i in range(batch_size):
            index = np.random.randint(0, len(self.text) - seq_length)
            seq.append(self.text[index:index+seq_length])
            next_char.append(self.text[index+seq_length])
            #print(seq)
            #print(next_char)
        return np.array(seq), np.array(next_char)

In [3]:
class RNN(tf.keras.Model):
    def __init__(self, num_chars, batch_size, seq_length):
        super().__init__()
        self.num_chars = num_chars
        self.seq_length = seq_length
        self.batch_size = batch_size
        self.cell = tf.keras.layers.LSTMCell(units=256)
        self.dense = tf.keras.layers.Dense(units=self.num_chars)
    def call(self, inputs, from_logits=False):
        inputs = tf.one_hot(inputs, depth=self.num_chars)
        state = self.cell.get_initial_state(batch_size=self.batch_size, dtype=tf.float32)
        #print("state", state)
        for t in range(self.seq_length):
            output, state = self.cell(inputs[:, t, :], state)
        logits = self.dense(output)
        if from_logits:
            return logits
        else:
            return tf.nn.softmax(logits)

In [4]:
num_batches = 1000
seq_length = 40
batch_size = 50
learning_rate = 1e-3

In [5]:
data_loader = DataLoader()

In [6]:
import numpy as np
model = RNN(num_chars=len(data_loader.chars), batch_size=batch_size, seq_length=seq_length)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(seq_length, batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 3.953150
batch 1: loss 3.936770
batch 2: loss 3.924057
batch 3: loss 3.889787
batch 4: loss 3.885556
batch 5: loss 3.831541
batch 6: loss 3.705526
batch 7: loss 3.404818
batch 8: loss 3.639490
batch 9: loss 2.982469
batch 10: loss 3.023395
batch 11: loss 2.954731
batch 12: loss 3.150671
batch 13: loss 3.294437
batch 14: loss 3.187977
batch 15: loss 3.001268
batch 16: loss 3.155298
batch 17: loss 3.202326
batch 18: loss 3.066057
batch 19: loss 2.820950
batch 20: loss 3.187222
batch 21: loss 3.047943
batch 22: loss 3.073235
batch 23: loss 3.024383
batch 24: loss 3.172982
batch 25: loss 3.311368
batch 26: loss 3.131812
batch 27: loss 3.131819
batch 28: loss 3.104218
batch 29: loss 2.991270
batch 30: loss 3.062727
batch 31: loss 2.886787
batch 32: loss 2.819685
batch 33: loss 3.159575
batch 34: loss 2.968798
batch 35: loss 2.966655
batch 36: loss 3.238174
batch 37: loss 3.368665
batch 38: loss 2.889974
batch 39: loss 3.345523
batch 40: loss 3.032537
batch 41: loss 3.147779
ba

In [7]:
def predict(self, inputs, temperature=1.):
    print("tf.shape", tf.shape(inputs))
    batch_size, _ = tf.shape(inputs)
    logits = self(inputs, from_logits=True)
    prob = tf.nn.softmax(logits / temperature).numpy()
    return np.array([np.random.choice(self.num_chars, p=prob[i, :]) for i in range(batch_size.numpy())])

In [9]:
model.predict = predict
X_, _ = data_loader.get_batch(seq_length, 1)
print("X_", X_)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    X = X_
    print("diversity %f:" % diversity)
    for t in range(400):
        y_pred = model.predict(X, diversity)
        print(data_loader.indices_char[y_pred[0]], end='', flush=True)
        X = np.concatenate([X[:, 1:], np.expand_dims(y_pred, axis=1)], axis=-1)
    print("\n")

X_ [[45  1 34 44  1 41 30 43 33 26 41 44  1 26 44  1 44 46 27 45 37 30  7  0
  27 30 37 26 45 30 29  7  1 29 34 47 30 43 44 30]]
diversity 0.200000:
tf.shape tf.Tensor([], shape=(0,), dtype=int32)


ValueError: not enough values to unpack (expected 2, got 0)